In [1]:
import sympy as sp
from msu_esd import cross_flow_unmixed, log_mean_temp_difference, parallel_single_pass, counter_single_pass

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 6333 & Homework 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 1
## Given
Water enters a counterflow, double-pipe heat exchanger at a rate of $70\,\frac{kg}{min}$ and is heated from $15^\circ C$ to $60^\circ C$ by an oil with a specific heat of $1.9\,\frac{kJ}{kg\,K}$. The oil enters at $116^\circ C$ and leaves at $27^\circ C$. The overall heat transfer coefficient is $300\,\frac{W}{m^2\,K}$

## Find
a. What heat transfer area is required?
b. What area is required if all conditions remain the same except that a shell and tube heat exchanger is used, with the water making one shell pass and the oil making two tube passes?
c. What exit water temperature would result if, for the exchanger of part (a), the water flow rate were decreased to $50\,\frac{kg}{min}$

## Solution
The specific heat of water will be taken at the average temperature of the water entrance and exit ($C_p=4.18\,\frac{kJ}{kg\,K}$)

### Part A
The oil is the hot fluid and the water is the cold fluid. The condition is unmixed because the fluids never meet.

In [2]:
# Declare constants as given
mc_ = 70
Cp_c_, Cp_h_ = 4.18, 1.9
Tc_in_, Tc_out_ = 15, 60
Th_in_, Th_out_ = 116, 27
U_ = 300

Cc, Ch, Th_out, Th_in, Tc_out, Tc_in = sp.symbols(r'C_c C_h T_{h\,out} T_{h\,in} T_{c\,out} T_{c\,in}')

# Solving for Ch
eq = sp.Eq(Cc*(Tc_out - Tc_in), Ch*(Th_in - Th_out))
eq

Eq(C_c*(-T_{c,in} + T_{c,out}), C_h*(T_{h,in} - T_{h,out}))

In [3]:
Ch_solved = sp.solve(eq, Ch)[0]
Ch_solved

C_c*(-T_{c,in} + T_{c,out})/(T_{h,in} - T_{h,out})

In [4]:
# Solving for Cc
Cc_ = mc_*Cp_c_
Cc_  # kJ per (min deg C)

292.59999999999997

In [5]:
Ch_ = Cc_*(Tc_out_ - Tc_in_)/(Th_in_ - Th_out_)
Ch_  # kJ per (min deg C)

147.94382022471908

In [6]:
C_min_ = min([Ch_, Cc_])
C_min_  # kJ per (min deg C)

147.94382022471908

In [7]:
# Get actual q
q_act_ = Cc_*(Tc_out_ - Tc_in_)
q_act_  # kJ per min

13166.999999999998

In [8]:
# Get q max
q_max_ = C_min_*(Th_in_ - Tc_in_)
q_max_  # kJ per min

14942.325842696626

In [9]:
# Effectiveness
epsilon_ = q_act_/q_max_
epsilon_

0.8811881188118813

In [10]:
# C
C_ = C_min_/Cc_
C_

0.5056179775280899

In [11]:
# Find the ntu value
ntu_ = cross_flow_unmixed(epsilon_, C_)
ntu_

4.180545804015178

The NTU relationship is,

$NTU=\frac{UA}{C_{min}}\rightarrow A=NTU\frac{C_{min}}{U}$

In [12]:
# Finding the area (unit manipulation added)
A_ = ntu_*C_min_/U_*1000/60
A_  # m^2

34.36032871502362

### LMTD Method

In [13]:
T_ = log_mean_temp_difference(Th_in_, Th_out_, Tc_in_, Tc_out_)
q_act_/(U_*T_)*1000/60

25.60989880574635

\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 6333 & Homework 2 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 2
## Given
The attributes of the inlet streams to a heat exchanger are,

| Stream | Inlet Temperature ($^\circ F$) | Capacity ($\frac{Btu}{hr\,^\circ F}$) |
|--------|--------------------------------|---------------------------------------|
| Hot    | 600                            | 50,000                                |
| Cold   | 500                            | 25,000                                |

The UA product in $\frac{Btu}{hr\,^\circ F}$ for the heat exchanger is given by,

$UA=\frac{1}{\frac{0.12}{C_h^{0.8}}+\frac{0.06}{C_c^{0.8}}+2\times10^{-7}}$

where $C_h$ and $C_c$ are the capacities in $\frac{Btu}{hr\,^\circ F}$.

## Find
Using only the $NTU$ method,

a. Find the outlet temperatures and rating for a parallel flow arrangement
b. Find the outlet temperatures and rating for a counter flow arrangement
c. Find the outlet temperatures and rating if two of these heat exchangers are placed in series and are operated in a parallel flow arrangement. What are the interface temperatures?
d. Find the outlet temperatures and rating if two of these heat exchangers are placed in series and operated in a counterflow arrangement. What are the interface temperatures?
e. Based ont he results of the above, discuss the utility of placing parallel and counterflow heat exchangers in series.

## Solution

In [14]:
# Define known constants with units mentioned in the given statement
Cc_ = 25_000
Ch_ = 50_000
Tc_in_ = 500
Th_in_ = 600

# Find the value for UA
# First a symbolic definition
UA = 1/(0.12/Ch**0.8 + 0.06/Cc**0.8 + 2e-7)
UA

1/(0.06/C_c**0.8 + 0.12/C_h**0.8 + 2.0e-7)

In [15]:
UA_ = float(UA.subs([(Cc, Cc_), (Ch, Ch_)]))
UA_  # In Btu per (hr*deg F)

25457.073923275435

In [16]:
# Find NTU
C_min_ = min([Cc_, Ch_])
NTU_ = UA_/C_min_
NTU_

1.0182829569310174

In [17]:
# Get the C_value
C_ = C_min_/max([Cc_, Ch_])
C_

0.5

### Part A
We know the $NTU$ value and $C$, which is enough to solve for the effectiveness.

In [18]:
# Get the effectiveness/rating
# Assuming a one pass parallel
parallel_effectiveness = parallel_single_pass(NTU_, C_, find='e')
parallel_effectiveness

0.5219372748457521

In [19]:
# Find q max
q_max_ = C_min_*(Th_in_ - Tc_in_)
q_max_

2500000

In [20]:
# Find the actual q
q_act_ = q_max_*parallel_effectiveness
q_act_  # In btu per hour

1304843.1871143803

In [21]:
# Find cold outlet temperature
Tc_out_ = q_act_/Cc_ + Tc_in_
Tc_out_  # deg F

552.1937274845752

In [22]:
# Find the hot outlet temperature
Th_out_ = Th_in_ - q_act_/Ch_
Th_out_  # deg F

573.9031362577124

### Part B
Everything is the same except the effectiveness correlation is different.

In [23]:
# Solving for the effectiveness/rating
counter_effectiveness = counter_single_pass(NTU_, C_, find='e')
counter_effectiveness

0.5703958472801851

In [24]:
q_act_ = q_max_*counter_effectiveness
q_act_  # Btu per hr

1425989.6182004628

In [25]:
# Find cold outlet temperature
Tc_out_ = q_act_/Cc_ + Tc_in_
Tc_out_  # deg F

557.0395847280186

In [26]:
# Find the hot outlet temperature
Th_out_ = Th_in_ - q_act_/Ch_
Th_out_  # deg F

571.4802076359907

### Part C

\begin{center}
\includegraphics{images/Capture}
\end{center}

The process is similar except now a $UA_{tot}$ should be implemented instead.

$UA_{tot}=\frac{1}{R_{tot}}$ \
$R_{tot}=R_1+R_2=\frac{1}{U_1A_1}+\frac{1}{U_2A_2}$

In [27]:
R_tot = 1/UA_ + 1/UA_
UA_tot_ = 1/R_tot
UA_tot_  # Btu per (hr*deg F)

12728.536961637717

In [28]:
# Find NTU
NTU_ = UA_tot_/C_min_
NTU_

0.5091414784655087

In [29]:
# Find the parallel effectiveness
series_parallel_flow = parallel_single_pass(NTU_, C_, find='e')
series_parallel_flow

0.3560442894721975

In [30]:
q_act_ = q_max_*series_parallel_flow
q_act_  # Btu per hr

890110.7236804938